PySpark Primer Notebook
By Kevin Chamberlin
Updated July 2024

In [1]:
%pip install fsspec --quiet
%pip install s3fs --quiet
%pip install pyspark --quiet

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

24/08/01 19:10:09 WARN Utils: Your hostname, codespaces-00e44c resolves to a loopback address: 127.0.0.1; using 10.0.1.4 instead (on interface eth0)
24/08/01 19:10:09 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/01 19:10:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# Example of manually creating a Spark DataFrame using PySpark code 
l_avengers_data = [[1, "Steve", "Rogers", "Brooklyn, NY", "Blue"]
              ,[2, "Tony", "Stark", "Manahattan, NY", "Gold"]
              ,[3, "Peter", "Parker", "Queens, NY", "Blue"]
              ,[4, "Scott", "Lang", "Coral Gables, FL", "Blue"]
              ,[5, "Natasha", "Romanoff", "Stalingrad, USSR", "Black"]
              ,[6, "Clint", "Barton", "Waverly, IA", "Purple"]]

l_avengers_col_names = ["ID", "FirstName", "LastName", "Hometown", "Favorite Color"]

sdf_avengers = spark.createDataFrame(l_avengers_data, l_avengers_col_names)

display(sdf_avengers)

DataFrame[ID: bigint, FirstName: string, LastName: string, Hometown: string, Favorite Color: string]

24/08/01 19:10:22 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors
